# Libraries

In [ ]:
import numpy as np
from keras.utils import to_categorical

# Functions

### Function that randomly deletes some digits from the solution

In [ ]:
def delete_digits(X, digits_to_delete=1):
    grids = X.argmax(3)
    for grid in grids:
        grid.flat[np.random.randint(0, 81, digits_to_delete)] = 0
    return to_categorical(grids)

### Function that sums up the difference between true solutions and predicted ones

In [ ]:
def total_diff(true, predicted):
    return np.sum(true != predicted, axis=(1, 2))

### Function that convert from grid to one-hot

In [ ]:
def to_one_hot(x): 
    return to_categorical(x).astype('float32')

### Function that convert from one-hot matrix to grid

In [1]:
def from_one_hot(one_hot_solution):
    return np.argmax(one_hot_solution, axis=2)

### Function that convert probability vector to one-hot array

In [ ]:
def proba_to_one_hot(proba):
    return np.array(proba == max(proba), dtype=np.float32)

### Function that generates prediction

In [1]:
def iterative_predict(puzzles, model):
    solutions = puzzles.copy()
    for solution in solutions:
        zeros = from_one_hot(solution) == 0
        while zeros.any():
            # prediction
            pred = model.predict(np.expand_dims(solution, axis=0))[0]
            
            # find the position of zeros
            zero_pos = np.where(zeros)
            zero_pos = list(zip(zero_pos[0], zero_pos[1]))
            
            # get best probability for each cell
            max_prob = np.max(pred, axis=2)
            
            # get the largest prob of all empty cells
            best_pos = zero_pos[0]
            best_prob = max_prob[best_pos]
            for pos in zero_pos:
                if max_prob[pos] > best_prob:
                    best_pos = pos
                    best_prob = max_prob[pos]
            
            # fill in the cell
            solution[best_pos] = np.append([0], proba_to_one_hot(pred[best_pos]))
            
            # update zeros
            zeros = from_one_hot(solution) == 0
    return solutions

### Function that check if a solution is correct

In [ ]:
def correct_solution(puzzle, solution):
    def is_sudoku_list(lst):
        return max(lst) == 9 and min(lst) == 1 and len(set(lst)) == len(lst)
    
    match_puzzle = (81 - np.sum(puzzle == solution)) == np.sum(puzzle == 0)
    if match_puzzle:
        bad_rows = [row for row in solution if not is_sudoku_list(row)]
        bad_cols = [col for col in solution.T if not is_sudoku_list(col)]
        bad_squares = []
        for i in np.arange(9, step=3):
            for j in np.arange(9, step=3):
                square = [solution[r][c] for r in range(i, i + 3) for c in range(j, j + 3)]
                if not is_sudoku_list(square):
                    bad_squares.append(square)
        return not (bad_rows or bad_cols or bad_squares)
    else:
        return False

### Function that create training puzzle from solution

In [ ]:
def to_puzzles(solutions):
    puzzles = solutions.copy()
    puzzles = [to_categorical(from_one_hot(puzzle) + 1).astype('float32')
               for puzzle in puzzles]
    return np.array(puzzles)